## Video Games Sales

`pip install pandas`

In [2]:
from datetime import datetime
import filecmp
import os
import pandas as pd

resources_path = "resources/"

vg_sales_path = resources_path + "vg_sales/"
vg_sales_res = {
    "v0": "vg_sales_v0.csv",
    "v1": "vg_sales_v1.csv",
    "extended_release_year": "vg_sales_extended_release_year.csv",
    "extended_release_year_2": "vg_sales_extended_release_year_2.csv",
    "filled_year": "vg_sales_filled_year.csv",
    "filled_year_2": "vg_sales_filled_year_2.csv",
}

vg_developers_path = resources_path + "vg_developers/"
vg_developers_res = {
    "v0": "vg_developers.csv",
}

results_path = "results/"

In [3]:
def save_to_csv(df, folder_path, filename):
    # Generate the full path for the new file
    new_file_path = os.path.join(folder_path, filename)
    
    # Check if the folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Save the dataframe to a temporary file
    temp_file_path = os.path.join(folder_path, 'temp_' + filename)
    df.to_csv(temp_file_path, index=True)
    
    # Compare the temporary file with existing files in the folder
    for existing_file in os.listdir(folder_path):
        existing_file_path = os.path.join(folder_path, existing_file)
        if existing_file_path != temp_file_path and filecmp.cmp(temp_file_path, existing_file_path, shallow=False):
            print(f"The content of {filename} already exists as {existing_file}. Not saving the file.")
            os.remove(temp_file_path)
            return
    
    # If no matching file is found, rename the temporary file to the new file
    os.rename(temp_file_path, new_file_path)
    print(f"File saved as {new_file_path}")

In [4]:
# Load the datasets with index

# vg_sales
vg_sales = pd.read_csv(vg_sales_path + vg_sales_res["v0"], index_col=0)

# vg_developers
vg_developers = pd.read_csv(vg_developers_path + vg_developers_res["v0"], index_col=0)
vg_developers = vg_developers.iloc[:, [0, 2, 3]]
vg_developers.rename(columns={'Est.': 'Year_of_Establishment'}, inplace=True)

In [5]:
# columns
print("vg_sales.columns :", vg_sales.columns)
print("vg_developers.columns :", vg_developers.columns)

vg_sales.columns : Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')
vg_developers.columns : Index(['City', 'Country', 'Year_of_Establishment'], dtype='object')


In [6]:
# NaN count
nan_count = vg_sales.isna().sum()
print(nan_count)

Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64


In [7]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_no_year = vg_sales[vg_sales['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_no_year, folder_path, filename)

The content of vg_sales_no_year_20241210_170408.csv already exists as vg_sales_no_year_20241210_162859.csv. Not saving the file.


In [8]:
# Load the extended release year data
vg_sales_extended = pd.read_csv(vg_sales_path + vg_sales_res["extended_release_year"], index_col=0)

# Merge the extended release year data with the original vg_sales dataframe
vg_sales = vg_sales.merge(vg_sales_extended[['Year_of_Release']], left_index=True, right_index=True, how='left', suffixes=('', '_extended'))

# Update the Year_of_Release column with the extended data where available
vg_sales['Year_of_Release'] = vg_sales['Year_of_Release_extended'].combine_first(vg_sales['Year_of_Release'])

# Drop the extended column as it's no longer needed
vg_sales.drop(columns=['Year_of_Release_extended'], inplace=True)

# Display the updated dataframe
print(vg_sales.head())

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_{timestamp}.csv'

# Save the updated dataframe to a separate CSV file
save_to_csv(vg_sales, folder_path, filename)

                         Platform  Year_of_Release         Genre Publisher  \
Name                                                                         
Wii Sports                    Wii           2006.0        Sports  Nintendo   
Super Mario Bros.             NES           1985.0      Platform  Nintendo   
Mario Kart Wii                Wii           2008.0        Racing  Nintendo   
Wii Sports Resort             Wii           2009.0        Sports  Nintendo   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

                          NA_Sales  EU_Sales  JP_Sales  Other_Sales  \
Name                                                                  
Wii Sports                   41.36     28.96      3.77         8.45   
Super Mario Bros.            29.08      3.58      6.81         0.77   
Mario Kart Wii               15.68     12.76      3.79         3.29   
Wii Sports Resort            15.61     10.93      3.28         2.95   
Pokemon Red/Pokemon Blue   

In [9]:
# vg_sales_filled_year
vg_sales_filled_year = pd.read_csv(vg_sales_path + vg_sales_res["filled_year"], index_col=0)

In [10]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_filled_year_no_year = vg_sales_filled_year[vg_sales_filled_year['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_filled_year_no_year, folder_path, filename)

The content of vg_sales_filled_year_no_year_20241210_170408.csv already exists as vg_sales_filled_year_no_year_20241210_162927.csv. Not saving the file.


In [11]:
# Load the extended release year data
vg_sales_extended_2 = pd.read_csv(vg_sales_path + vg_sales_res["extended_release_year_2"], index_col=0)

# Merge the extended release year data with the original vg_sales dataframe
vg_sales = vg_sales.merge(vg_sales_extended_2[['Year_of_Release']], left_index=True, right_index=True, how='left', suffixes=('', '_extended'))

# Update the Year_of_Release column with the extended data where available
vg_sales['Year_of_Release'] = vg_sales['Year_of_Release_extended'].combine_first(vg_sales['Year_of_Release'])

# Drop the extended column as it's no longer needed
vg_sales.drop(columns=['Year_of_Release_extended'], inplace=True)

# Display the updated dataframe
print(vg_sales.head())

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_2'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_2_{timestamp}.csv'

# Save the updated dataframe to a separate CSV file
save_to_csv(vg_sales, folder_path, filename)

                         Platform  Year_of_Release         Genre Publisher  \
Name                                                                         
Wii Sports                    Wii           2006.0        Sports  Nintendo   
Super Mario Bros.             NES           1985.0      Platform  Nintendo   
Mario Kart Wii                Wii           2008.0        Racing  Nintendo   
Wii Sports Resort             Wii           2009.0        Sports  Nintendo   
Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

                          NA_Sales  EU_Sales  JP_Sales  Other_Sales  \
Name                                                                  
Wii Sports                   41.36     28.96      3.77         8.45   
Super Mario Bros.            29.08      3.58      6.81         0.77   
Mario Kart Wii               15.68     12.76      3.79         3.29   
Wii Sports Resort            15.61     10.93      3.28         2.95   
Pokemon Red/Pokemon Blue   

In [ ]:
# vg_sales_filled_year_2
vg_sales_filled_year_2 = pd.read_csv(vg_sales_path + vg_sales_res["filled_year_2"], index_col=0)

In [13]:
# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_filled_year_2_no_year'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Isolate rows where Year_of_Release is NaN
vg_sales_filled_year_2_no_year = vg_sales_filled_year_2[vg_sales_filled_year_2['Year_of_Release'].isna()]

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_filled_year_2_no_year_{timestamp}.csv'

# Save to a separate CSV file
save_to_csv(vg_sales_filled_year_2_no_year, folder_path, filename)

The content of vg_sales_filled_year_2_no_year_20241210_170408.csv already exists as vg_sales_filled_year_2_no_year_20241210_165438.csv. Not saving the file.


### Cancelled games in the dataset
Brothers in Arms: Furious 4,X360,,Shooter,,0.01,0.0,0.0,0.0,0.01,,,,,Gearbox Software,M
was never released and put into sales

In [14]:
# Remove the specified entry
vg_sales_cleaned_release_date = vg_sales_filled_year_2.drop(index='Brothers in Arms: Furious 4') # type: ignore

# Create a folder in results_path if it doesn't exist
folder_name = 'vg_sales_cleaned_release_date'
folder_path = os.path.join(results_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Generate a unique filename based on the current date and time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'vg_sales_cleaned_release_date_{timestamp}.csv'

# Save the cleaned dataframe to a separate CSV file
save_to_csv(vg_sales_cleaned_release_date, folder_path, filename)

File saved as results/vg_sales_cleaned_release_date/vg_sales_cleaned_release_date_20241210_170408.csv


In [15]:
# vg_sales_v1
vg_sales_v1 = pd.read_csv(vg_sales_path + vg_sales_res["v1"], index_col=0)

In [19]:
# Isolate rows where Year_of_Release is NaN
vg_sales_v1_no_release_date = vg_sales_v1[vg_sales_v1['Year_of_Release'].isna()]
print("Number of video games that have no release date :", len(vg_sales_v1_no_release_date))

Number of video games that have no release date : 0
